You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document:

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention of the latest trends and advancements in AI technology

Key Points:
1. Latest Trends in AI:
- Machine learning and deep learning advancements
- Natural language processing applications
- AI in healthcare, finance, and transportation sectors
- Ethical considerations in AI development

2. Key Players in the

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

**Introduction:**

Artificial Intelligence (AI) has revolutionized various industries with its innovative technologies, shaping the way we live and work. From machine learning to natural language processing, AI continues to push boundaries and drive advancements across sectors. The latest trends in AI are constantly evolving, with breakthroughs in healthcare, finance, transportation, and more. It is essential to explore the key players in the AI industry and stay updated on the noteworthy news and developments in this rapidly growing field.

**Latest Trends in AI:**

Machine learning and deep learning are at the forefront of AI advancements, enabling systems to learn from data and make intelligent decisions. Natural language processing applications have also seen significant growth, with AI-powered chatbots and voice assistants transforming customer interactions. In sectors like healthcare, AI is revolutionizing patient care with predictive analytics and personalized treatment plans. However, ethical considerations in AI development, such as bias and privacy concerns, remain crucial for ensuring responsible AI applications.

**Key Players in the AI Industry:**

Leading the pack in AI innovation are companies like Google DeepMind, IBM Watson, Amazon Web Services, and Microsoft Azure. These tech giants are driving AI research and development, with a focus on creating cutting-edge solutions for businesses and consumers. Google DeepMind, known for its AlphaGo program, continues to push the boundaries of AI in gaming and strategic decision-making. IBM Watson's cognitive computing capabilities are transforming industries like healthcare and finance, while Amazon Web Services and Microsoft Azure offer scalable AI solutions for cloud computing and data analytics.

**Noteworthy News in AI:**

Recent breakthroughs in AI research have showcased the potential of AI to revolutionize industries and drive economic growth. From self-driving cars to automated financial trading systems, AI implementations are reshaping businesses and government agencies worldwide. However, ethical considerations and regulatory frameworks are essential for ensuring the responsible development and deployment of AI technologies. Stay informed on the latest AI news and updates to understand the implications of AI on society and the economy.

**Conclusion:**

As AI continues to shape the future of technology and innovation, staying informed on the latest trends, key players, and news in the AI industry is essential. By exploring the advancements in machine learning, natural language processing, and AI applications across sectors, we can better understand the potential of AI to drive positive change. Embracing ethical considerations and regulatory frameworks in AI development is crucial for ensuring the responsible and sustainable growth of AI technologies. Stay connected with industry newsletters, AI events, and reputable online sources to stay ahead of the curve in the dynamic world of artificial intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Agentic Frameworks"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Agentic Frameworks.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: Agentic Frameworks

Outline:
I. Introduction
- Definition of Agentic Frameworks
- Importance of understanding Agentic Frameworks in today's society

II. Latest Trends in Agentic Frameworks
- Rise of digital agentic tools
- Incorporation of agentic principles in leadership development

III. Key Players in Agentic Frameworks
- Theoretical foundations from Bandura's social cognitive theory
- Organizations implementing agentic frameworks in their operations

IV. Noteworthy News on Agentic Frameworks
- Cas

In [14]:
Markdown(result)

# Understanding Agentic Frameworks: Empowering Personal and Organizational Growth

## Introduction
Agentic frameworks play a vital role in today's society, representing the belief in one's capability to influence their own motivation, behavior, and social surroundings. The comprehension of agentic frameworks is fundamental for individuals and organizations striving for personal growth and empowerment. By grasping the core principles of agentic frameworks, individuals can enhance their leadership skills, while organizations can elevate employee engagement and performance.

## Latest Trends in Agentic Frameworks
One of the current trends in agentic frameworks is the emergence of digital tools that empower individuals to take charge of their personal development. These tools offer tailored feedback, goal-setting features, and progress monitoring capabilities, all designed to nurture a sense of agency in users. Furthermore, agentic principles are now integrated into leadership development programs, highlighting the significance of self-efficacy, goal-setting, and resilience in effective leadership.

## Key Players in Agentic Frameworks
Agentic frameworks are deeply rooted in Bandura's social cognitive theory, underscoring the importance of self-regulation, self-efficacy, and observational learning in human behavior. Organizations are progressively embracing agentic frameworks in their operations to empower employees, enhance decision-making processes, and cultivate a culture of innovation. By embracing the principles of agentic frameworks, organizations can establish a more vibrant and adaptable work environment.

## Noteworthy News on Agentic Frameworks
Case studies demonstrating successful implementation of agentic frameworks illustrate the positive impact on individual and organizational outcomes. Research findings reveal that organizations prioritizing employee empowerment and autonomy witness heightened levels of job satisfaction, productivity, and creativity. Through the implementation of agentic frameworks, organizations can unleash the full potential of their employees and drive sustainable growth.

## Target Audience Analysis
Professionals in psychology, leadership development, and organizational behavior can derive significant benefits from understanding agentic frameworks and applying them in their practice. Individuals keen on personal growth and empowerment can leverage agentic principles to steer their own development and achieve their objectives. Organizations aiming to enhance employee engagement and performance can harness agentic frameworks to establish a more positive and fruitful work environment.

## Conclusion
In conclusion, agentic frameworks provide a robust foundation for personal and organizational growth. By incorporating SEO keywords like agentic frameworks, Bandura's social cognitive theory, empowerment in organizations, and leadership development with agentic principles, we can broaden our reach and establish ourselves as a reliable source of information in this domain. For further insights on leveraging agentic frameworks for personal and organizational growth, subscribe to our newsletter for regular updates and exclusive content.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).